In [ ]:
from numpy import *
import numpy as np
%config InlineBackend.rc = {}
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
from astropy.io import fits
from astropy.nddata import Cutout2D
import scipy
from scipy import stats
from scipy.optimize import curve_fit

In [ ]:
bb_list = ['hw3_fits/bright-bb650-5s-'+n+'.fits' for n in ['1','2','3']]
bias_list = ['hw3_fits/bias-'+n+'.fits' for n in ['1','2','3']]

bb_concat = []
for image in bb_list:
    bb_concat.append(fits.getdata(image))
    
bias_concat = []
for image in bias_list:
    bias_concat.append(fits.getdata(image))

blackbody_stacked = np.sum(bb_concat, axis=0)
bias_stacked = np.sum(bias_concat, axis=0)
image_data = blackbody_stacked-bias_stacked

In [ ]:
cutout = Cutout2D(image_data, (501.04,392.04),(90,90))
cropped = cutout.data
print('Mean:', np.mean(cutout.data)/3)
print('Stdev:', np.std(cutout.data))

To determine read noise in ADU:
Subtract two bias frames, find standard deviation then divide by square root of two

In [ ]:
bias1 = fits.getdata('hw3_fits/bias-2.fits')
bias2 = fits.getdata('hw3_fits/bias-3.fits')
bias_diff = bias1-bias2
cutout = Cutout2D(bias_diff, (501.04,392.04),(90,90))
bias_diff_c = cutout.data
diffh = reshape(bias_diff_c, 8100)

print('When entire sample is considered, read noise is:',np.std(bias_diff_c)/sqrt(2))
"""
I found that about half of the values were low integer values, 
and the other half was >65000, I'm not sure what defect would cause this
but I assumed it was a systematic error that could be removed
"""
print(sort(diffh))
print(sort(diffh)[4300:4400])
i = np.where(diffh > 65000)
diffh[i] -= 65500
s = sort(diffh)
image_hist = plt.hist(diffh.flatten(), 500)

std_bias = np.std(s)
readnoise = std_bias/sqrt(2)
print(readnoise)

"""
Alternatively, splitting the array into two parts
"""
bias_l = []
bias_g = []
i = np.where(bias_diff_c > 65000)
bias_g.extend(bias_diff_c[i])
k = np.where(bias_diff_c < 65000)
bias_l.extend(bias_diff_c[k])

readnoise_l=np.std(bias_l)/sqrt(2)
readnoise_g=np.std(bias_g)/sqrt(2)
print(readnoise_l,readnoise_g)

Calculating dark current at different temperatures, code refactored from code developed for an astronomy lab at University of Iowa (description - http://homepage.physics.uiowa.edu/~pkaaret/2013f_29c137/Lab03_noise.html, code - http://homepage.physics.uiowa.edu/~pkaaret/2013f_29c137/darktime.py) 

In [ ]:
cutout_bias = Cutout2D(bias_stacked, (501.04,392.04),(100,100))
bias = cutout_bias.data
#bias = bias_stacked


# array of temp
temp = array([0.0,-10.0,-23.5,10.0,20.0])
temp_string = ['-00.0C','-10.0C','-23.5C','p10.0C','p20.0C']
# list of dark files
darkfile = ['hw3_fits/dark100'+t+'.fits' for t in temp_string]


# arrays to hold results
c_mean = 0.0*temp
c_median = 0.0*temp
c_rms = 0.0*temp

# wait after each set of plots
plt.ioff()
colmap = plt.get_cmap('gray') # load gray colormap

# process the files
for i in range(len(darkfile)):
  # read in the file
  image_data = fits.getdata(darkfile[i])
  cutout = Cutout2D(image_data, (501.04,392.04),(100,100))
  h = cutout.data 
  #h = image_data
  print('Read ', darkfile[i])
  # copy the image data into a numpy (numerical python) array
  img = h
  # find the difference fromthe bias frames
  diff = img - bias
  # diff is a 2-d array, need to change to 1-d to make a histogram
  nx, ny = diff.shape # find the size of the array
  diffh = reshape(diff, nx*ny) # change the shape to be 1d
  smin = int(len(diffh)*0.03)
  smax = int(len(diffh)-smin)
  s = sort(diffh)[smin:smax] 
  print('len',len(diffh))
  
  vmin = np.min(s)
  vmax = np.max(s)
  print('bounds = ', vmin, vmax)
  # one figure with two plots
  plt.figure(1)
  # plot the first image
  plt.subplot(121)
  plt.imshow(diff, cmap=colmap, vmin=vmin, vmax=vmax) # plot image using gray colorbar
  # plot the second image
  plt.subplot(122)
  # plot a histogram of the image values
  ht = plt.hist(diffh, bins=vmax-vmin, range=[vmin,vmax], histtype='stepfilled', color='k')
  nc = max(ht[0]) # maximum value in plotted histogram
  # select only values within ranges
  q = where((diffh >= vmin) & (diffh <= vmax))
  diffhcut = diffh[q]
  # find statistics
  c_mean[i] = mean(diffhcut)
  c_median[i] = median(diffhcut)
  c_rms[i] = std(diffhcut)
  print('mean, median, rms = ', c_mean[i], c_median[i], c_rms[i])
  plt.plot([c_mean[i], c_mean[i]], [0, nc], '-g')
  plt.plot([c_median[i], c_median[i]], [0, nc], '--g')
  plt.show() # display the plots
  

# plot median versus time
m = c_mean
#m = c_median
plt.figure(2)
plt.xlabel('Temp (c)')
plt.ylabel('Mean Counts')
#plt.ylabel('Median Counts')
plt.plot(temp, m, 'd')
scipy.optimize.curve_fit
# do a linear fit to the data
slope, intercept, r_value, p_value, std_err = stats.linregress(temp, m)
print('slope = ', slope)
print('intercept = ', intercept)
print('correlation coefficient r =', r_value)
# plot the fit
plt.plot(temp, intercept + temp*slope, '--r')
plt.show()



In [ ]:
temp=[10,20]
m=[4330.7,13302.3]
# do a linear fit to the data
slope, intercept, r_value, p_value, std_err = stats.linregress(temp, m)
print('slope = ', slope)
print('intercept = ', intercept)
print('correlation coefficient r =', r_value)



In [ ]:
noise = sqrt(36140+8.3**2+45**2)
g = 34815/noise**2
print(noise,g)